In [1]:
import os
# Change to directory where vmtk_functions and vmtk_filenames are located
os.chdir('/home/iagolessa/Documents/aneurysms/vmtkScripts')

In [4]:
%%writefile vmtk_functions.py

from vmtk import vmtkscripts
import numpy as np

# Collection of function for some usual tasks

# Image reader
def readImage(imageFileName):
    """ Function to read image VTK object from disk.
        The function returns an image VTK object. 
        
        Input arguments: 
        - imageFileName (str): string containing the
                         image filename with full path;
        
        Output: vtkImageObject 
    """
    # Instantiation of object image reader
    imageReader = vmtkscripts.vmtkImageReader()
    # Input members
    imageReader.InputFileName = imageFileName
    imageReader.Execute()
    
    return imageReader.Image 


def readDicom(dicomDirPath,dicomFilePrefix):
    """ Funtion to read a DICOM directory and 
        returns a vtkImageData. The file prefix 
        can be added, if not must be specified as
        'None'.
        
        Input arguments:
        - dicomDirPath (str): a string with the path to DICOM image files
        - dicomFilePrefix (str or None): prefix of DICOM first file (without extension).
            if the DICOM dir contains only one type of file prefix, then 
            use 'None'
            
        Output: vtkImageObject
    """
    # Instantiating image reader object
    imageReader = vmtkscripts.vmtkImageReader()

    # Try to read directly from directory
    # If not, read from first file
    if dicomFilePrefix is not None:
        imageReader.InputFileName = dicomDirPath+dicomFilePrefix
        imageReader.Execute()
    else:
        imageReader.Format = 'dicom'
        imageReader.InputDirectoryName = dicomDirPath
        imageReader.Execute()
    
    return imageReader.Image



# Image viewer function
def viewImage(image):
    """ Function for visualize VTK image objects.
    
        Input arguments:
        - vtkImageObject: the image to be displayed.
    
        Output: renderer displaying vtkImageData.
    """
    imageViewer = vmtkscripts.vmtkImageViewer()
    imageViewer.Image = image
    imageViewer.Margins = 1
    imageViewer.Execute()

# View 3D image with opacity
def viewImageMIP(image):
    """ Function for visualize VTK image objects
        with MIP visualization (internally, uses
        the vmtkimagemipviewer script)
    
        Input arguments:
        - vtkImageData: the image to be displayed.
    
        Output: 3D renderization displaying vtkImageData
        in a MIP projection.
    """
    imageViewer = vmtkscripts.vmtkImageMIPViewer()
    imageViewer.Image = image
    imageViewer.Execute()

    
# Writing a loaded vtkImageData to disk 
def writeImage(image,imageOutputFileName):
    """ Function that writes a vtkImageData to disk,
        given the vtkImageObject and an output file name. 
        
        Input arguments:
        - vtkImageData: image object containing the image
                        to be written.
        
        Output: file stored at imageOutputFileNale
    """
    
    # Writing image to file
    imageWriter = vmtkscripts.vmtkImageWriter()
    imageWriter.Image = image
    imageWriter.OutputFileName = imageOutputFileName
    imageWriter.Execute()

    
    
# Surface reader
def readSurface(surfaceFileName):
    """ Function to read surface VTK object from disk.
        The function returns a surface VTK object. 
        
        Input arguments: 
        - surfaceFileName (str): string containing the
                                 surface filename with full path;
        
        Output: vtkPolyData object 
     """
    surfaceReader = vmtkscripts.vmtkSurfaceReader()
    surfaceReader.InputFileName = surfaceFileName
    surfaceReader.Execute()
    
    return surfaceReader.Surface


# Viewing surface
def viewSurface(surface):
    """ Function for visualize VTK surface objects.
    
        Input arguments:
        - vtkPolyData: the surface to be displayed.
    
        Output: renderer displaying vtkPolyData.
    """
    surfaceViewer = vmtkscripts.vmtkSurfaceViewer()
    surfaceViewer.Surface = surface
    surfaceViewer.Execute()

    
# Writing a surface
def writeSurface(surface,fileName,mode):
    """ Function that writes a surface to disk,
        given the vtkPolyData and an output file name. 
        
        Input arguments:
        - vtkPolyData: poly data object containing the 
                       surface to be written.
        - fileName (str): a string containing the file name.
        - mode (ascii,binary): mode to be written.
        
        Output: file stored at fileName.
    """
    surfaceWriter = vmtkscripts.vmtkSurfaceWriter()
    surfaceWriter.Surface = surface
    surfaceWriter.OutputFileName = fileName
    
    if mode == 'ascii' or mode == 'binary':
        surfaceWriter.Mode = mode
    
    surfaceWriter.Execute()
    


# Using marching cubes to extract an initial surface
def initialSurface(image,level):
    """ Creates an initial preliminary surface using 
        the Marching Cubes algorithm and extract the 
        largest connected region.
        
        Input arguments: 
        - vtkImageData: a VTK image object;
        - level (float): the level to pass to Marching Cubes 
        
        Output: a vtkPolyData (surface) corresponding to
                that level of the image
    """
    surfaceMarchingCubes = vmtkscripts.vmtkMarchingCubes()

    # The input image to Marching Cubes must be the Level Sets image
    surfaceMarchingCubes.Image = image

    # Parameters
    surfaceMarchingCubes.Level = level
    surfaceMarchingCubes.Execute()
    
    # Run vmtksurfacetriangle to prevent pathed surfaces from Marching Cubes
    # before using the vmtksurfaceconnected
    surfaceTriangle = vmtkscripts.vmtkSurfaceTriangle()
    surfaceTriangle.Surface = surfaceMarchingCubes.Surface
    surfaceTriangle.Execute()

    # Extract largest connected surface
    surfaceConnected = vmtkscripts.vmtkSurfaceConnectivity()
    surfaceConnected.Surface = surfaceTriangle.Surface
    surfaceConnected.Execute()
    
    return surfaceConnected.Surface



def viewCenterline(centerline,array):
    """Function to visualize a centerline with or without ids."""
    centerlinesViewer = vmtkscripts.vmtkCenterlineViewer()
    
    listOfArrays = ["CenterlineIds",
                    "TractIds",
                    "GroupIds",
                    "Blanking",
                    "MaximumInscribedSphereRadius"]
    
    centerlinesViewer.Centerlines = centerline
    
    if array in listOfArrays:
        centerlinesViewer.CellDataArrayName = array
    else:
        print("Array name not known")
    
    centerlinesViewer.Execute()
    


# Functions that encapsulate the smoothing and subdivider operations
# because they may be used on other parts of the procedure

# Smooth surface
def smoothSurface(surface):
    """ Surface smoother based on Taubin or Laplace algorithm.
        
        Input arguments:
        - vtkPolyData: the surface object to be smoothed;
        
        Output: vtkPolyData
    """
    
    # Instantiation of vmtk script object
    surfaceSmoother = vmtkscripts.vmtkSurfaceSmoothing()
    
    # Smoothing parameters
    surfaceSmoother.Surface = surface
    surfaceSmoother.Method = 'taubin'
    surfaceSmoother.NumberOfIterations = 30
    surfaceSmoother.PassBand = 0.1
    
    #surfaceSmoother.PrintInputMembers()
    #surfaceSmoother.PrintOutputMembers()
    
    # Smooth
    surfaceSmoother.Execute()
    return surfaceSmoother.Surface


# Subdivide triangles
def subdivideSurface(surface):
    """ Surface triangle subdivider based 
        on Taubin or Laplace algorithm.
        
        Input arguments:
        - vtkPolyData: the surface object to be subdivided;
        
        Output: vtkPolyData
    """
    surfaceSubdivide = vmtkscripts.vmtkSurfaceSubdivision()

    surfaceSubdivide.Surface = surface
    surfaceSubdivide.Method = 'butterfly' # or linear
    surfaceSubdivide.Execute()
    
    return surfaceSubdivide.Surface


# NEEDS TO BE TESTED YET!
# Function that receives a surface with open inlets and outlets and returns 
# Numpy array with patch geometrical infos, such as center point, outward normal, patch id and patch type
def getPatchInfo(surface):
    """ Function that generates a Numpy structured ndarray
        with geometrical and type information of open
        inlets and outlets of a vessel network surface.
        The array is structured as follows:
        
        [((xCenter1, yCenter1, zCenter1), (xNormal1, yNormal1, zNormal1), radius1, id1, type),
         ((xCenter2, yCenter2, zCenter2), (xNormal2, yNormal2, zNormal2), radius2, id2, type),
         ...
         ((xCenterN, yCenterN, zCenterN), (xNormalN, yNormalN, zNormalN), radiusN, idN, type)]
         
        The function defines the inlet as the section with largest radius, and accepts 
        vessel geometries with only one flow inlet.
        
        Input arguments:
        - Input surface (vtkPolyData): surface to be analyzed;
        
        Output arguments:
        - Numpy structured ndarray.
    """
    # Type of each columns
    colsType = [('Center', tuple),
                ('Normal', tuple),
                ('Radius', float),
                ('PatchId', int),
                ('PatchType','U10')]
    
    # Empty auxiliar list     
    capsGeometryList = []
    
    # Capping the surface
    surfaceCapper = vmtkscripts.vmtkSurfaceCapper()
    surfaceCapper.Surface = surface
    surfaceCapper.Method = 'centerpoint'
    surfaceCapper.Interactive = 0
    surfaceCapper.Execute()

    # Surface boundary inspector
    # Conveting surface to mesh to get cell entity ids
    surfaceToMesh = vmtkscripts.vmtkSurfaceToMesh()
    surfaceToMesh.Surface = surfaceCapper.Surface
    surfaceToMesh.Execute()

    # Inspecting
    surfaceBoundaryInspector = vmtkscripts.vmtkMeshBoundaryInspector()
    surfaceBoundaryInspector.Mesh = surfaceToMesh.Mesh
    surfaceBoundaryInspector.CellEntityIdsArrayName = surfaceCapper.CellEntityIdsArrayName
    surfaceBoundaryInspector.Execute()

    # Store patch info in python dictionary using vmtksurfacetonumpy
    vmtkToNumpy = vmtkscripts.vmtkSurfaceToNumpy()
    vmtkToNumpy.Surface = surfaceBoundaryInspector.ReferenceSystems
    vmtkToNumpy.Execute()

    dictPatchData = vmtkToNumpy.ArrayDict
    # Creation of a 'capsGeometryArray' ndarray structured object
    # which contains Center, Normals, Radius, and patch type
    # of the surface caps

    # Assigning structuired array with patch info
    intPatchesNum = len( dictPatchData['CellData']['CellPointIds'] )

    for index in range(intPatchesNum):
        # Copy formatted to list
        Center  = tuple(dictPatchData['Points'][index])
        Normal  = tuple(dictPatchData['PointData']['BoundaryNormals'][index])
        PatchId = dictPatchData['PointData']['CellEntityIds'][index]
        Radius  = dictPatchData['PointData']['BoundaryRadius'][index]

        capsGeometryList.append((Center, Normal, Radius, PatchId, 'patch'))

    # Convert to array
    capsGeometryArray = np.array(capsGeometryList, 
                                 dtype=colsType)

    # Change patch type based on maximum radius
    maxRadius = np.max(capsGeometryArray['Radius'])

    for i in range(0, len(capsGeometryArray)):
        if  capsGeometryArray['Radius'][i] == maxRadius:
            capsGeometryArray['PatchType'][i] = 'inlet'
        else:
            capsGeometryArray['PatchType'][i] = 'outlet'

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Specify the other inlets here
#     if inletIds is not None:
#         extraInletIds = inletIds

#         # For all the patches ids
#         for patchId in capsGeometryArray['PatchId']:
#             # If the id is an extraInlet id
#             # Check if extraInletIds is not empty
#             if extraInletIds:
#                 # If the patch id is on specified inlet list
#                 if patchId in extraInletIds:
#                     # find its index in capsGeometryArray
#                     index = np.where(capsGeometryArray['PatchId'] == patchId)
#                     capsGeometryArray['PatchType'][index[0]] = 'inlet'
#                 # If not, then name it 'outlet'
#                 else:
#                     index = np.where(capsGeometryArray['PatchId'] == patchId)
#                     capsGeometryArray['PatchType'][index[0]] = 'outlet'
#             else:
#                 continue

    return capsGeometryArray

Overwriting vmtk_functions.py


In [5]:
%%writefile vmtk_filenames.py


# Extensions
imageExtension   = ".vti"
surfaceExtension = ".vtp"
meshExtension    = ".vtu"

# Default File Names
# images
imageDicomFile     = 'imageOriginal.vti'
imageVoiFile       = 'imageVoi.vti'
imageLevelSetsFile = 'imageLevelSets.vti'
imageInitialLevelSetsFile = 'imageInitialLevelSets.vti'

# surfaces
surfaceInputFile       = 'surfaceVoi.vtp'
surfaceDicomFile       = 'surfaceDicom.vtp'
surfaceSmoothedFile    = 'surfaceSmoothed.vtp'
surfaceVoiFile         = 'surfaceVoi.vtp'
surfaceVoiSmFile       = 'surfaceVoiSm.stl'
surfaceClippedFile     = 'surfaceClipped.vtp'
surfaceEndClippedFile  = 'surfaceEndClipped.vtp'
surfaceRemeshedFile    = 'surfaceRemeshed.vtp'
surfaceRemeshedSmFile  = 'surfaceRemeshedSm.stl'
surfaceCappedFile      = 'surfaceCapped.vtp'
surfaceOrientedFile    = 'surfaceOriented.vtp'
surfaceWithFlowExtFile = 'surfaceWithFlowExt.vtp'
surfaceDistToCenterlinesFile = 'surfaceDistToCenterlines.vtp'
surfaceFinalFile       = 'surfaceFinal.vtp' 
# centerlines
centerlineWithoutAneurysm = 'centerlineWithoutAneurysm.vtp'
centerlineWithAneurysm = 'centerlineWithAneurysm.vtp'
centerlineSectionsFile = 'centerlineSections.vtp'
centerlinesAttributesFile = 'centerlinesAttributes.vtp'
centerlineGeometryFile = 'centerlineGeometry.vtp'

DelaunayTesselationFile = 'DelaunayTesselation.vtp'
VoronoiDiagramFile = 'VoronoiDiagram.vtp'

# other
referenceSystemsInitialOrientationFile = 'referenceSystemsInitialOrientation.dat'
referenceSystemsOrientedFile = 'referenceSystemsOriented.dat'

Overwriting vmtk_filenames.py


In [16]:
def extractRawSurface(image, level, inflation):
    """ Function to extract a raw surface from
        a DICOM image (or other format). The image 
        passed must be loaded in memory. The surface 
        is extracted from a level and inflated by the
        parameter passed of the Marching cubes algorithm.
        
        Input args:
        - Input image (vtkImageData): the input image
        - level (float): the level to be extracted from the
            the original image
        - inflation (float): how much to inflate final surface
            to coincide with vessel surface.
        - write (bool): to whether or not to save files 
            with resulting level sets image and marching
            cubes surgace
        
        Output:
        - Surface (vtkPolyData): the final extracted surface.
        """
    # Object instantiantion
    initializationImage = vmtkscripts.vmtkImageInitialization()
    
    initializationImage.Image = image
    initializationImage.Method = 'isosurface'
    initializationImage.IsoSurfaceValue = level
    initializationImage.Interactive = 0
    #initializationImage.InitialLevelSetsOutputFileName = imagesDir+imageInitialLevelSetsFile
    initializationImage.Execute()
    
    # Debug
    # initializationImage.PrintInputMembers()
    # initializationImage.PrintOutputMembers()

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
    # If image was generated from dicom above
    imageLevelSets = vmtkscripts.vmtkLevelSetSegmentation()

    # Input image
    imageLevelSets.Image = initializationImage.Image
    imageLevelSets.InitialLevelSets = initializationImage.InitialLevelSets
    
    # Once the branch segmented, the parameters that have led to good agreement between the extracted level and image are:
    # -- NumberOfIterations = 300
    # -- PropagationScaling = 0.5: usually 0.0, but the isosurface method does not get the gradients very well
    # -- CurvatureScaling = 0.1: usually 0.0, it is important not to set very high because it can degenerate the surface
    #                            however, i noted that setting slighty above 0.0 remove high curvature parts on the surface
    # -- AdvectionScaling = 1.0: usually 1.0 is perfect

    imageLevelSets.NumberOfIterations = 300
    imageLevelSets.PropagationScaling = 0.5
    imageLevelSets.CurvatureScaling = 0.1
    imageLevelSets.AdvectionScaling = 1.0
    imageLevelSets.SmoothingIterations = 20

    # Execute Level Sets
    imageLevelSets.Execute()

    # Debug
    # imageLevelSets.PrintInputMembers()
#     imageLevelSets.PrintOutputMembers()

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
    # Converting VOI image to surface using Marching Cubes
    surfaceMarchingCubes = vmtkscripts.vmtkMarchingCubes()

    # The input image to Marching Cubes must be the Level Sets image
    surfaceMarchingCubes.Image = imageLevelSets.LevelSets

    # Parameters
    surfaceMarchingCubes.Level = inflation # increasing the value to inflate the surface
    surfaceMarchingCubes.Execute()

    # Run vmtksurfacetriangle to prevent pathed surfaces from Marching Cubes
    # before using the vmtksurfaceconnected
    surfaceTriangle = vmtkscripts.vmtkSurfaceTriangle()
    surfaceTriangle.Surface = surfaceMarchingCubes.Surface
    surfaceTriangle.Execute()

    # Extract largest connected surface
    surfaceConnected = vmtkscripts.vmtkSurfaceConnectivity()
    surfaceConnected.Surface = surfaceTriangle.Surface
    surfaceConnected.Execute()
    
    return surfaceConnected.Surface

In [7]:
# Function to correct surface with holes and glued parts
def fixSurface(surface):
    """ Function to interactively fix a surface of a 
        vessel segment with holes and other weird 
        artifacts. The function works internally with 
        the region drawing script and clipper with
        array. It is important to do not input a surface
        with an existing field defined on it.
        
        The remesh operation is particularly long if the
        surface is large. We therefore recommend to cut the 
        surface before with only the 'region of interest'.
        
        Input args:
        - Input surface (vtkPolyData): the surface to be edited
        
        Output: 
        - Surface (vtkPolyData)
    """

    cutRegionsMarker = vmtkscripts.vmtkSurfaceRegionDrawing()
    cutRegionsMarker.Surface = surface
    cutRegionsMarker.Execute()

    # Debug
    # cutRegionsMarker.PrintInputMembers()
    # cutRegionsMarker.PrintOutputMembers()

    
    # # Since the array created by RegionDrawing is discontinous
    # # we use the array smoothing
    # cutRegionsSmoother = vmtkscripts.vmtkSurfaceArraySmoothing()
    # cutRegionsSmoother.Surface = cutRegionsMarker.Surface
    # cutRegionsSmoother.SurfaceArrayName = cutRegionsMarker.ContourScalarsArrayName
    # # General options
    # cutRegionsSmoother.Connexity = 1
    # cutRegionsSmoother.Relaxation = 1.0
    # cutRegionsSmoother.Iterations = 10
    # cutRegionsSmoother.Execute()

    # Initial Clipper to revome excess of surface
    surfaceArrayClipper = vmtkscripts.vmtkSurfaceClipper()

    surfaceArrayClipper.Surface = cutRegionsMarker.Surface
    surfaceArrayClipper.Interactive = 0
    surfaceArrayClipper.ClipArrayName = cutRegionsMarker.ContourScalarsArrayName #cutRegionsSmoother.SurfaceArrayName
    surfaceArrayClipper.ClipValue = 0.5*(cutRegionsMarker.InsideValue + \
                                         cutRegionsMarker.OutsideValue)
    surfaceArrayClipper.Execute()

    # Debug
    # surfaceArrayClipper.PrintInputMembers()
    # surfaceArrayClipper.PrintOutputMembers()

    # Extract largest connected surface
    surfaceConnected = vmtkscripts.vmtkSurfaceConnectivity()
    surfaceConnected.Surface = surfaceArrayClipper.Surface
    surfaceConnected.Execute()

    # Simple remesh procedure to increase quality at cut lines
    surfaceRemesh = vmtkscripts.vmtkSurfaceRemeshing()

    surfaceRemesh.Surface = surfaceConnected.Surface
    surfaceRemesh.ElementSizeMode = "edgelength"
    surfaceRemesh.TargetEdgeLength = 0.2
    surfaceRemesh.SurfaceOutputFileName = surfacesDir+surfaceRemeshedFile
    surfaceRemesh.Execute()
    surfaceRemesh.IOWrite()
    # surfaceRemesh.Execute()

    surfaceFixer = vmtkscripts.vmtkSurfaceCapper()

    surfaceFixer.Surface = surfaceRemesh.Surface
    # Capping Method = 'simple', 'centerpoint', 'smooth', 'annular', 'concaveannular'
    surfaceFixer.Method = 'smooth'
    surfaceFixer.ConstraintFactor = 0.2
    surfaceFixer.NumberOfRings = 6
    surfaceFixer.Interactive = 0

    surfaceFixer.Execute()

    # Debug
    # surfaceFixer.PrintInputMembers()
    # surfaceFixer.PrintOutputMembers()
    return surfaceFixer.Surface

In [6]:
def extractAneurysm(surface):
    """ Function to extract an aneurysm surface 
        by selecting its neck with a drawing 
        tool.
        
        Input args:
        - input surface (vtkPolyData): VTK surface object 
            of a vessels tree with an aneurysm.
            
        Output:
        - Aneurysm surface (vtkPolyData).
    """
    
    aneurysmNeckDraw = vmtkscripts.vmtkSurfaceRegionDrawing()
    aneurysmNeckDraw.Surface = surface
    aneurysmNeckDraw.Execute()

    # Debug
    # aneurysmNeckDraw.PrintInputMembers()
    # aneurysmNeckDraw.PrintOutputMembers()
    
    # Since the array created by RegionDrawing is discontinous
    # we use the array smoothing
    aneurysmNeckSmoother = vmtkscripts.vmtkSurfaceArraySmoothing()
    aneurysmNeckSmoother.Surface = aneurysmNeckDraw.Surface
    aneurysmNeckSmoother.SurfaceArrayName = aneurysmNeckDraw.ContourScalarsArrayName
    # General options
    aneurysmNeckSmoother.Connexity = 1
    aneurysmNeckSmoother.Relaxation = 1.0
    aneurysmNeckSmoother.Iterations = 20
    aneurysmNeckSmoother.Execute()

    # Debug
    # aneurysmNeckSmoother.PrintInputMembers()
    # aneurysmNeckSmoother.PrintOutputMembers()

    # Surface clipper
    aneurysmClipper = vmtkscripts.vmtkSurfaceClipper()

    aneurysmClipper.Surface = aneurysmNeckSmoother.Surface
    aneurysmClipper.Interactive = 0
    aneurysmClipper.ClipArrayName = aneurysmNeckDraw.ContourScalarsArrayName
    aneurysmClipper.ClipValue = 0.5*(aneurysmNeckDraw.InsideValue + \
                                     aneurysmNeckDraw.OutsideValue)
    aneurysmClipper.Execute()

    # Debug
    # aneurysmClipper.PrintInputMembers()
    # aneurysmClipper.PrintOutputMembers()

    # Simple remesh procedure to increase quality at cut lines
    # aneurysmRemesh.Surface = aneurysmClipper.Surface
    # aneurysmRemesh.ElementSizeMode = "edgelength"
    # aneurysmRemesh.Execute()
    return aneurysmClipper.Surface

Testing

In [2]:
from vmtk import vmtkscripts
import numpy as np
import os

In [3]:
# Path where files are stored
aneurysmsPath = "/home/iagolessa/documents/unesp/doctorate/"

In [4]:
# Change to directory where vmtk_functions and vmtk_filenames are located
os.chdir(aneurysmsPath+"aneurysms/vmtkScripts")

# Import the necessary user's modules
import vmtk_functions
from vmtk_filenames import *

# Change it back
os.chdir('/home/iagolessa')

In [5]:
# Path to save the generated files when extracting the surface
# We reccomend to separate it from the DICOM original dir. 
# Note that inside casePath, this code will create different sub-directories to store different data types
casePath       = aneurysmsPath+"aneurysms/geometries/vrCases/extraCases/imageSubtracted/AquilionPrime/newImage/"

# Define subdirs
imagesDir       = casePath+'images/'
surfacesDir     = casePath+'surfaces/'
centerlinesDir  = casePath+'centerlines/'
meshesDir       = casePath+'meshes/'
parentVesselDir = casePath+"parentVessel/"

print('Files saved to:', casePath)

Files saved to: /home/iagolessa/documents/unesp/doctorate/aneurysms/geometries/vrCases/extraCases/imageSubtracted/AquilionPrime/newImage/


In [23]:
imageDicom = vmtk_functions.readImage("/home/iagolessa/documents/unesp/master/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case18/images/case18preStent.vti")

Reading VTK XML image file.

Spacing 0.495744 0.495744 0.495743

Origin -0.000000 -0.000000 -126.414726

Dimensions 256 256 256


In [24]:
vmtk_functions.viewImage(imageDicom)

Quit renderer


In [25]:
surface = extractRawSurface(imageDicom,200,0.1)

Isosurface initialization.

Progress: 100%Progress: 100%


In [26]:
vmtk_functions.viewSurface(surface)

Quit renderer


In [18]:
surface = vmtk_functions.smoothSurface(surface)

In [20]:
# Clipping the surface
roiSelector = vmtkscripts.vmtkSurfaceClipper()

roiSelector.Surface = surface
# roiSelector.SurfaceInputFileName = surfacesDir+surfaceRemeshedSmFile
# roiSelector.IORead()
roiSelector.InsideOut = 1
roiSelector.WidgetType = 'box'
roiSelector.Execute()

Quit renderer


In [21]:
surface = fixSurface(surface)

Quit renderer

Writing VTK XML surface file.
